In [ ]:
from pyspark.sql import SparkSession, SQLContext
from creds import USERNAME as UNAME
from creds import PASSWORD as PASS

URL = "jdbc:mysql://localhost:3306/main3"

spark = SparkSession \
    .builder \
    .appName("Database access example") \
    .config('spark.driver.extraClassPath', './mysql-connector-java-8.0.16.jar') \
    .getOrCreate()
# TODO: research the .config options


sc = spark.sparkContext
sql_context = SQLContext(sc)
sc.setLogLevel("INFO")


def get_table(tablename):
    """
    Loads in a dataframe from the database using the tablename
    """
    return sql_context.read.format("jdbc").options(
    url=URL,
    dbtable = tablename,
    user=UNAME,
    password=PASS).load()

def execute_query(sql_query):
    """
    Executes an arbitrary sql statement and returns the dataframe that is loaded into memory
    """
    # TODO: make generic for each type of clause in a query?
    return sql_context.read.format("jdbc").options(
    url=URL,
    user=UNAME,
    password=PASS,
    query = sql_query, numPartitions=4).load()


def get_post_site_text(start_date, end_date):
    """
    Gets the site name and body of all posts within the specified date range 
    """
    # TODO: figure out what columns we actually want to select from here
    query = "SELECT site.name, post.body " + \
    "FROM main2.site INNER JOIN main2.post ON site.siteId = post.siteId " + \
    "WHERE createdDateTime BETWEEN \"" + \
    start_date + "\" AND \""+ end_date + "\""
    # string formatting could make more readable
    return execute_query(query)

def get_post_count_per_site(start_date, end_date):
    """
    returns a dataframe containing [name, count(*)] 
    where name is the site name and count is the number of posts on that site
    within the specified date range
    """
    # TODO - alias Count(*) as post_count
    query = "SELECT site.name, Count(*)" + \
    "FROM main2.site INNER JOIN main2.post ON site.siteId = post.siteId " + \
    "WHERE createdDateTime BETWEEN \"" + \
    start_date + "\" AND \""+ end_date + "\"" + \
    "GROUP BY site.siteId"
    return execute_query(query)

def get_answer_count_per_site(start_date, end_date):
    """
    returns a dataframe containing [name, Count(*)] 
    where name is the site name and count is the number of answered questions on that site
    within the specified date range
    
    """
    query = "SELECT site.name, Count(*)" + \
    "FROM main2.site INNER JOIN main2.post ON site.siteId = post.siteId " + \
    "INNER JOIN main2.answer ON answer.u_postId = post.u_postId " + \
    "WHERE createdDateTime BETWEEN \"" + \
    start_date + "\" AND \""+ end_date + "\"" + \
    "GROUP BY site.siteId"
    return execute_query(query)

def get_avg_len_posts(start_date, end_date):
    """
    returns a dataframe containing [name, AVG(length(post.body))] 
    where name is the site name and avg is average length of a post 
    within the specified date range
    """
    # TODO - alias the averages
    # TODO - should this just be questions or just answers?
    query = "SELECT site.name, AVG(length(post.body)) " + \
    "FROM main2.site INNER JOIN main2.post ON site.siteId = post.siteId " + \
    "WHERE createdDateTime BETWEEN \"" + \
    start_date + "\" AND \""+ end_date + "\"" +\
    "GROUP BY site.siteId"
    return execute_query(query) 

def get_avg_len_comments(start_date, end_date):
    """
    returns a dataframe containing [name, AVG(length(comment.body))] 
    where name is the site name and avg is average length of a comment 
    within the specified date range
    """
    query = "SELECT site.name, AVG(length(comment.body)) " + \
    "FROM main2.site INNER JOIN main2.comment ON site.siteId = comment.siteId " + \
    "WHERE createdDateTime BETWEEN \"" + \
    start_date + "\" AND \""+ end_date + "\"" +\
    "GROUP BY site.siteId"
    return execute_query(query) 

def get_total_size_posts(start_date, end_date):
    """
    returns a dataframe containing [name, SUM(length(post.body))] 
    where name is the site name and sum is the total length of all posts
    within the specified date range
    """
    query = "SELECT site.name, SUM(length(post.body))" + \
    "FROM main2.site INNER JOIN main2.post ON site.siteId = post.siteId " + \
    "WHERE createdDateTime BETWEEN \"" + \
    start_date + "\" AND \""+ end_date + "\"" + \
    "GROUP BY site.siteId"
    return execute_query(query)

def get_total_size_comments(start_date, end_date):
    """
    returns a dataframe containing [name, SUM(length(comment.body))] 
    where name is the site name and count is the total length of all comments
    within the specified date range
    """
    query = "SELECT site.name, SUM(length(comment.body))" + \
    "FROM main2.site INNER JOIN main2.comment ON site.siteId = comment.siteId " + \
    "WHERE createdDateTime BETWEEN \"" + \
    start_date + "\" AND \""+ end_date + "\"" + \
    "GROUP BY site.siteId"
    return execute_query(query)

def main():
    get_post_site_text("2018-03-15", "2018-03-16").show()
    get_post_count_per_site("2018-03-15", "2018-03-16").show()
    get_answer_count_per_site("2018-03-15", "2018-03-16").show()
    get_avg_len_posts("2018-03-15", "2018-03-16").show()
    get_avg_len_comments("2018-03-15", "2018-03-16").show()
    get_total_size_posts("2018-03-15", "2018-03-16").show()
    get_total_size_comments("2018-03-15", "2018-03-16").show()

main()



In [ ]:
spark.sparkContext._conf.getAll() 